In [1]:
import pandas as pd
from geopy import distance
import requests
import json
import datetime
import math
import itertools
import numpy as np

In [2]:
df = pd.read_csv("concap.csv")

In [3]:
df.rename(columns={"CountryName": "Country", "CapitalName": "capital", "CapitalLatitude": "lat", "CapitalLongitude": "lon", "CountryCode": "code", "ContinentName": "continent"}, inplace=True)
df

,Country,capital,lat,lon,code,continent
0,Somaliland,Hargeisa,9.550000,44.050000,NaN,Africa
1,South Georgia and South Sandwich Islands,King Edward Point,-54.283333,-36.500000,GS,Antarctica
2,French Southern and Antarctic Lands,Port-aux-Français,-49.350000,70.216667,TF,Antarctica
3,Palestine,Jerusalem,31.766667,35.233333,PS,Asia
4,Aland Islands,Mariehamn,60.116667,19.900000,AX,Europe
...,...,...,...,...,...,...
240,Northern Cyprus,North Nicosia,35.183333,33.366667,NaN,Europe
241,Hong Kong,NaN,0.000000,0.000000,HK,Asia
242,Heard Island and McDonald Islands,NaN,0.000000,0.000000,HM,Antarctica
243,British Indian Ocean Territory,Diego Garcia,-7.300000,72.400000,IO,Africa


In [4]:
ropa = df[df["capital"].isin(["Rome","Paris"])].reset_index()
cities = ropa.copy()
cities

,index,Country,capital,lat,lon,code,continent
0,81,France,Paris,48.866667,2.333333,FR,Europe
1,110,Italy,Rome,41.900000,12.483333,IT,Europe


In [5]:
d = distance.distance((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
d, d.km, d.miles

(Distance(1107.8818760940028), 1107.8818760940028, 688.4058822066647)

In [6]:
getattr(d, "km")

1107.8818760940028

In [7]:
results = []
for f in [distance.distance, distance.great_circle, distance.geodesic]:
    for mes in ["kilometers","km","miles","mi","nautical","nm","feet","ft"]:
        d = f((cities.loc[0, "lat"], cities.loc[0, "lon"]), (cities.loc[1, "lat"], cities.loc[1, "lon"]))
        results.append({"method": f.__name__, "measurement": mes, "value": getattr(d, mes)})

# show as dataframe
results_df = pd.DataFrame(results)
results_df.pivot_table(index="method", columns="measurement", values="value")

measurement,feet,ft,kilometers,km,mi,miles,nautical,nm
method,,,,,,,,
geodesic,3.634783e+06,3.634783e+06,1107.881876,1107.881876,688.405882,688.405882,598.208356,598.208356
great_circle,3.630457e+06,3.630457e+06,1106.563205,1106.563205,687.586498,687.586498,597.496331,597.496331


In [8]:
cities = ropa.copy()
r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{cities.loc[0, "lon"]},{cities.loc[0, "lat"]};{cities.loc[1, "lon"]},{cities.loc[1, "lat"]}?overview=false""")

In [9]:
r.content

b'{"code":"Ok","waypoints":[{"hint":"6pvUj____39RAAAAYQAAAAkAAAAAAAAAUTdZQvgsIUGPZrZAAAAAAFEAAABhAAAACQAAAAAAAACy5AAA45kjAJ6l6QKVmiMAa6XpAgEAvwyxLYfu","distance":14.236102,"location":[2.333155,48.866718],"name":"Rue Saint-Roch"},{"hint":"MDXNh____38VAAAAZgAAAAAAAAAAAAAA3AxzQf1MYEIAAAAAAAAAABUAAABmAAAAAAAAAAAAAACy5AAAbXu-AFBXfwIFe74A4Fd_AgAAHw-xLYfu","distance":18.175415,"location":[12.483437,41.899856],"name":"Via dell\'Umilt\xc3\xa0"}],"routes":[{"legs":[{"steps":[],"weight":54542,"distance":1432278.8,"summary":"","duration":54515.9}],"weight_name":"routability","weight":54542,"distance":1432278.8,"duration":54515.9}]}'

In [10]:
json.loads(r.content)

{'code': 'Ok',
 'waypoints': [{'hint': '6pvUj____39RAAAAYQAAAAkAAAAAAAAAUTdZQvgsIUGPZrZAAAAAAFEAAABhAAAACQAAAAAAAACy5AAA45kjAJ6l6QKVmiMAa6XpAgEAvwyxLYfu',
   'distance': 14.236102,
   'location': [2.333155, 48.866718],
   'name': 'Rue Saint-Roch'},
  {'hint': 'MDXNh____38VAAAAZgAAAAAAAAAAAAAA3AxzQf1MYEIAAAAAAAAAABUAAABmAAAAAAAAAAAAAACy5AAAbXu-AFBXfwIFe74A4Fd_AgAAHw-xLYfu',
   'distance': 18.175415,
   'location': [12.483437, 41.899856],
   'name': "Via dell'Umiltà"}],
 'routes': [{'legs': [{'steps': [],
     'weight': 54542,
     'distance': 1432278.8,
     'summary': '',
     'duration': 54515.9}],
   'weight_name': 'routability',
   'weight': 54542,
   'distance': 1432278.8,
   'duration': 54515.9}]}

In [11]:
route_1 = json.loads(r.content)["routes"][0]
route_1["distance"], route_1["duration"]

(1432278.8, 54515.9)

In [12]:
x = datetime.timedelta(seconds=route_1["duration"])
str(x)

'15:08:35.900000'

In [13]:
df1 = pd.read_csv("Base1.csv")

In [14]:
df1

,Left,Right
0,35.582066,-78.802120
1,35.582066,-78.801965
2,35.582010,-78.799760
3,35.581969,-78.798268
4,35.581968,-78.796966
5,35.584332,-78.798292
6,35.585262,-78.798268
7,35.582924,-78.799764
8,35.583693,-78.799788
9,35.584345,-78.799816


In [15]:
r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{df1.loc[0, "Right"]},{df1.loc[0, "Left"]};{df1.loc[14, "Right"]},{df1.loc[14, "Left"]}?overview=false""")

In [16]:
json.loads(r.content)

{'code': 'Ok',
 'waypoints': [{'hint': 'yEaWislGlooAAAAAFAAAAAAAAAAAAAAAAAAAAAnOYEEAAAAAAAAAAAAAAAAUAAAAAAAAAAAAAADMRAAAOJNN-3LwHgI4k037cvAeAgAAnwUEyBLO',
   'distance': 0,
   'location': [-78.80212, 35.582066],
   'name': 'West Vance Street'},
  {'hint': '80aWigFHlooAAAAAoQAAAAAAAAAAAAAAAAAAAMvo30IAAAAAAAAAAAAAAAChAAAAAAAAAAAAAADMRAAAQqJN-yzzHgJCok37LPMeAgAAnwUEyBLO',
   'distance': 0,
   'location': [-78.79827, 35.582764],
   'name': 'Raleigh Street'}],
 'routes': [{'legs': [{'steps': [],
     'weight': 65.1,
     'distance': 437.7,
     'summary': '',
     'duration': 65.1}],
   'weight_name': 'routability',
   'weight': 65.1,
   'distance': 437.7,
   'duration': 65.1}]}

In [17]:
import pandas as pd
from geopy import distance
import requests # to call the openmap/google apis
import json
import datetime
import math
import itertools
import numpy as np

In [18]:
df1 = pd.read_csv("Base1.csv")

In [19]:
r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{df1.loc[0, "Right"]},{df1.loc[0, "Left"]};{df1.loc[14, "Right"]},{df1.loc[14, "Left"]}?overview=false""")

In [20]:
json.loads(r.content)

{'code': 'Ok',
 'waypoints': [{'hint': 'yEaWislGlooAAAAAFAAAAAAAAAAAAAAAAAAAAAnOYEEAAAAAAAAAAAAAAAAUAAAAAAAAAAAAAADMRAAAOJNN-3LwHgI4k037cvAeAgAAnwUEyBLO',
   'distance': 0,
   'location': [-78.80212, 35.582066],
   'name': 'West Vance Street'},
  {'hint': '80aWigFHlooAAAAAoQAAAAAAAAAAAAAAAAAAAMvo30IAAAAAAAAAAAAAAAChAAAAAAAAAAAAAADMRAAAQqJN-yzzHgJCok37LPMeAgAAnwUEyBLO',
   'distance': 0,
   'location': [-78.79827, 35.582764],
   'name': 'Raleigh Street'}],
 'routes': [{'legs': [{'steps': [],
     'weight': 65.1,
     'distance': 437.7,
     'summary': '',
     'duration': 65.1}],
   'weight_name': 'routability',
   'weight': 65.1,
   'distance': 437.7,
   'duration': 65.1}]}

In [21]:
type(r)

requests.models.Response

In [22]:
r

<Response [200]>

In [23]:
type(r.content)

bytes

In [24]:
json.loads(r.content)["routes"][0]

{'legs': [{'steps': [],
   'weight': 65.1,
   'distance': 437.7,
   'summary': '',
   'duration': 65.1}],
 'weight_name': 'routability',
 'weight': 65.1,
 'distance': 437.7,
 'duration': 65.1}

In [25]:
json.loads(r.content)["routes"][0]["distance"]

437.7

In [26]:
df1

,Left,Right
0,35.582066,-78.802120
1,35.582066,-78.801965
2,35.582010,-78.799760
3,35.581969,-78.798268
4,35.581968,-78.796966
5,35.584332,-78.798292
6,35.585262,-78.798268
7,35.582924,-78.799764
8,35.583693,-78.799788
9,35.584345,-78.799816


In [27]:
for i in df1.index:
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [28]:
for i in df1.index:
    r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{df1.loc[0, "Right"]},{df1.loc[0, "Left"]};{df1.loc[i, "Right"]},{df1.loc[i, "Left"]}?overview=false""")
    print(json.loads(r.content)["routes"][0]["distance"])

0
14
213.9
349.2
467.1
611.6
715.6
314
395.3
473.7
113.8
298.7
214.8
119
437.7


In [29]:
df2 = pd.DataFrame()

In [30]:
first = []
for i in df1.index:
    r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{df1.loc[0, "Right"]},{df1.loc[0, "Left"]};{df1.loc[i, "Right"]},{df1.loc[i, "Left"]}?overview=false""")
    first.append(json.loads(r.content)["routes"][0]["distance"])

In [31]:
type(first)

list

In [32]:
df2[0] = first

In [33]:
df2

,0
0,0.0
1,14.0
2,213.9
3,349.2
4,467.1
5,611.6
6,715.6
7,314.0
8,395.3
9,473.7


In [34]:
df3 = pd.DataFrame()

In [35]:
for j in df1.index:
    list1 = []
    for i in df1.index:
        r = requests.get(f"""http://router.project-osrm.org/route/v1/car/{df1.loc[j, "Right"]},{df1.loc[j, "Left"]};{df1.loc[i, "Right"]},{df1.loc[i, "Left"]}?overview=false""")
        list1.append(json.loads(r.content)["routes"][0]["distance"])
    df3[j] = list1

In [36]:
df3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,14.0,213.9,349.2,467.1,612.1,715.6,315.6,401.1,473.7,113.8,298.7,214.8,119.0,437.7
1,14.0,0.0,199.9,335.2,453.0,598.1,701.5,301.5,387.1,459.7,99.8,284.7,200.8,105.0,423.7
2,213.9,199.9,0.0,135.3,253.2,397.7,501.1,101.7,187.2,259.8,100.1,283.9,200.8,301.2,217.7
3,349.2,335.2,135.3,0.0,117.8,262.8,366.3,237.0,322.6,395.1,235.4,420.3,336.4,440.2,88.4
4,467.1,453.0,253.2,117.8,0.0,377.9,480.1,354.8,440.4,512.9,353.2,538.1,454.2,558.0,201.6
5,611.6,597.5,397.7,262.8,380.7,0.0,103.5,296.0,210.4,137.9,497.8,303.5,387.4,484.0,174.4
6,715.6,701.5,501.7,366.3,484.1,103.5,0.0,399.5,313.9,241.3,601.7,407.0,490.9,587.5,277.9
7,314.0,299.9,101.7,237.0,354.8,296.0,399.5,0.0,85.6,158.1,200.1,183.0,99.2,199.6,154.1
8,395.3,381.3,187.2,322.6,440.4,210.4,313.9,85.6,0.0,72.6,281.5,96.6,180.5,280.9,239.6
9,473.7,459.7,259.8,395.1,512.9,137.9,241.3,158.1,72.6,0.0,359.9,165.7,257.3,357.7,312.2


In [37]:
df3.to_csv('file1.csv')